In [1]:
import torch
import collections

In [2]:
class DenseLayer():
  def __init__(self, features, neurons):
    self.weight = 0.01 * torch.rand(neurons, features)
    self.biases = torch.zeros(1, neurons)
  def forward(self, inputs):
    self.output = torch.matmul(inputs, self.weight.T) + self.biases

## Relu Activation Function

In [3]:
class ActivationRelu:

  def forward(self, inputs):
    output = torch.max(inputs, torch.tensor(0.0))
    return output


## Softmax activation function

In [4]:
class Activation_SoftMax:
  def forward(self, inputs):
    #calculate powers
    power_x = torch.exp(inputs)
    #get shape
    shape_x = inputs.shape
    sum_x = torch.sum(power_x, axis = 1, keepdims = True)
    #divide
    result = power_x / sum_x
    return result


## Sigmoid activation function

In [5]:
class Activation_Sigmoid:
  def forward(self, inputs):
    shape_x = inputs.shape
    ones = torch.full(shape_x, 1)
    sum_x = ones + inputs
    res = ones / sum_x
    return res


## Initialize activations

In [6]:
relu = ActivationRelu()
sigmoid = Activation_Sigmoid()
softmax = Activation_SoftMax()

## Model with relu activation function

In [8]:
manual_seed = 42
torch.manual_seed(manual_seed)

# Number of features
features = 4
# Neurons in input layers
neuron_1, neuron_2, neuron_3 = 18, 18, 18
# Output class
output_class = 3
# Number of samples
samples = 10


lower_bound = 0
upper_bound = 10000
input = (upper_bound - lower_bound) * torch.rand(samples, features) + lower_bound

layer_1 = DenseLayer(features, neuron_1)
layer_1.forward(input)
output_1 = relu.forward(layer_1.output)
print(output_1.shape)

layer_2 = DenseLayer(output_1.shape[1], neuron_2)
layer_2.forward(output_1)
output_2 = relu.forward(layer_2.output)
print(output_2.shape)


output_layer = DenseLayer(output_2.shape[1], output_class)
output_layer.forward(output_2)
print(output_layer.output.shape)
final_output_1 = softmax.forward(output_layer.output)
print(final_output_1)
print(final_output_1.shape)

torch.Size([10, 18])
torch.Size([10, 18])
torch.Size([10, 3])
tensor([[0.3159, 0.3673, 0.3168],
        [0.3223, 0.3547, 0.3230],
        [0.3164, 0.3649, 0.3188],
        [0.3171, 0.3639, 0.3190],
        [0.3201, 0.3593, 0.3206],
        [0.3233, 0.3525, 0.3242],
        [0.3275, 0.3439, 0.3286],
        [0.3217, 0.3540, 0.3243],
        [0.3186, 0.3621, 0.3193],
        [0.3192, 0.3599, 0.3208]])
torch.Size([10, 3])


## Model with sigmoid activation function

In [9]:
manual_seed = 42
torch.manual_seed(manual_seed)

# Number of features
features = 4
# Neurons in input layers
neuron_1, neuron_2, neuron_3 = 18, 18, 18
# Output class
output_classes = 3
# Number of samples
samples = 10


input = torch.rand(samples, features)

layer_1 = DenseLayer(features, neuron_1)
layer_1.forward(input)
output_1 = sigmoid.forward(layer_1.output)
print(output_1.shape)

layer_2 = DenseLayer(output_1.shape[1], neuron_2)
layer_2.forward(output_1)
output_2 = sigmoid.forward(layer_2.output)
print(output_2.shape)


output_layer = DenseLayer(output_2.shape[1], output_classes)
output_layer.forward(output_2)
final_output_2 = softmax.forward(output_layer.output)
print(final_output_2)
print(final_output_2.shape)

torch.Size([10, 18])
torch.Size([10, 18])
tensor([[0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323],
        [0.3318, 0.3359, 0.3323]])
torch.Size([10, 3])


In [10]:
# Generate random values for y_true (one-hot encoded)
y_true = torch.eye(output_classes)[torch.randint(output_classes, size=(samples,))]

# Generate random values for y_true_label (label-encoded)
y_true_label = torch.randint(output_classes, size=(samples,))
y_true, y_true_label

(tensor([[0., 1., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [1., 0., 0.]]),
 tensor([2, 1, 0, 2, 1, 1, 0, 2, 1, 2]))

# Calculating log loss for label encoding methods

In [15]:
def log_loss(y_true, y_pred):
    if y_true.dim() > 1:  # Check if y_true is one-hot encoded
        loss = -torch.mean(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))
    else:  # Integer-encoded labels
        y_true_one_hot = torch.zeros_like(y_pred)
        y_true_one_hot.scatter_(1, y_true.view(-1, 1), 1)
        loss = -torch.mean(y_true_one_hot * torch.log(y_pred) + (1 - y_true_one_hot) * torch.log(1 - y_pred))

    return loss

loss_1 = log_loss(y_true, final_output_1)
loss_2 = log_loss(y_true, final_output_2)
print("First model log loss: " , loss_1)
print("Second model log loss: ", loss_2)

First model log loss:  tensor(0.6377)
Second model log loss:  tensor(0.6368)


## calculating log loss for one-hot encoding methods

In [16]:
loss_1_label = log_loss(y_true, final_output_1)
loss_2_label = log_loss(y_true, final_output_2)
print("First model log loss: " , loss_1_label)
print("Second model log loss: ", loss_2_label)

First model log loss:  tensor(0.6377)
Second model log loss:  tensor(0.6368)
